In [1]:
from google.colab import files
files.upload()

import os
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json


In [2]:
# ========== 3. download and unzip the dataset ==========
!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset
License(s): MIT


In [3]:
# Check if the file exists before attempting to open it
import zipfile
import os
if os.path.exists("fashion-product-images-dataset.zip"):
    with zipfile.ZipFile("fashion-product-images-dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("fashion_data")
else:
    print("Error: 'fashion-product-images-dataset.zip' not found. Please ensure the download was successful.")



In [4]:
import pandas as pd
df = pd.read_csv("/content/fashion_data/fashion-dataset/fashion-dataset/styles.csv", on_bad_lines='skip')

In [5]:
# Get unique masterCategory labels
unique_labels = df['season'].dropna().unique()

# Count of unique labels
num_labels = len(unique_labels)

print(f"Number of unique season labels: {num_labels}")
print("Labels:", unique_labels)

Number of unique season labels: 4
Labels: ['Fall' 'Summer' 'Winter' 'Spring']


In [7]:
import os
import shutil
from tqdm import tqdm
import pandas as pd


filtered_df = df[df['season'].notna()].copy()


root_dir = "fashion_data_season"
os.makedirs(root_dir, exist_ok=True)


season_labels = filtered_df['season'].astype(str).unique()
for label in season_labels:
    os.makedirs(os.path.join(root_dir, label), exist_ok=True)


source_dir = "/content/fashion_data/fashion-dataset/images"
for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
    img_name = row['id']
    label = str(row['season'])

    src = os.path.join(source_dir, f"{img_name}.jpg")
    dst = os.path.join(root_dir, label)
    dst = os.path.join(dst, f"{img_name}.jpg")

    if os.path.exists(src):
        try:
            shutil.copy(src, dst)
        except Exception as e:
            print(f"❌ Error copying {img_name}: {e}")
    else:
        print(f"⚠️ Image not found: {src}")


 16%|█▌        | 7034/44403 [00:07<00:24, 1522.34it/s]

⚠️ Image not found: /content/fashion_data/fashion-dataset/images/39403.jpg


 37%|███▋      | 16302/44403 [00:15<00:27, 1040.23it/s]

⚠️ Image not found: /content/fashion_data/fashion-dataset/images/39410.jpg


 73%|███████▎  | 32423/44403 [00:32<00:11, 1065.99it/s]

...

 82%|████████▏ | 36445/44403 [00:39<00:14, 542.55it/s]

⚠️ Image not found: /content/fashion_data/fashion-dataset/images/39425.jpg


 90%|█████████ | 40026/44403 [00:46<00:10, 410.42it/s]

⚠️ Image not found: /content/fashion_data/fashion-dataset/images/12347.jpg


100%|██████████| 44403/44403 [01:00<00:00, 735.34it/s]


In [8]:
import torch
import torch.nn as nn
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load EfficientNet with 4 output classes
model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=4)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [9]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


full_dataset = datasets.ImageFolder("/content/fashion_data_season", transform=transform)


total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size


train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [11]:
import torch
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10, class_names=None):
    best_f1 = 0.0

    for epoch in range(epochs):
        # ---- Training ----
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        # ---- Save Best Model ----
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"✅ Best model updated at Epoch {epoch+1} | F1: {f1:.4f}")

        # ---- Confusion Matrix ----
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names if class_names else np.unique(all_labels),
                    yticklabels=class_names if class_names else np.unique(all_labels))
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title(f'Confusion Matrix - Epoch {epoch+1}')
        plt.tight_layout()
        plt.savefig(f'confusion_matrix_epoch_{epoch+1}.png')
        plt.close()

        # ---- Logging ----
        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | F1 Score: {f1:.4f}")


In [ ]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=10
)


✅ Best model updated at Epoch 1 | F1: 0.7146
Epoch 1 | Train Loss: 0.8851 | Val Loss: 2.0460 | F1 Score: 0.7146
✅ Best model updated at Epoch 2 | F1: 0.7536
Epoch 2 | Train Loss: 0.5796 | Val Loss: 0.6517 | F1 Score: 0.7536
✅ Best model updated at Epoch 3 | F1: 0.7701
Epoch 3 | Train Loss: 0.4776 | Val Loss: 0.6739 | F1 Score: 0.7701
✅ Best model updated at Epoch 4 | F1: 0.7786
Epoch 4 | Train Loss: 0.3723 | Val Loss: 0.6434 | F1 Score: 0.7786
✅ Best model updated at Epoch 5 | F1: 0.7882
Epoch 5 | Train Loss: 0.2749 | Val Loss: 0.6547 | F1 Score: 0.7882


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F
import copy

In [ ]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='macro')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (macro): {f1:.4f}")

In [ ]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (weighted): {f1:.4f}")